In [2]:
#### random fill data to train and validation folder with 80% ratio ######## 

## data from kagglecatsanddogs_3367a 


import os
import shutil
from random import sample



# Define paths
base_dir = './data_one/PetImages'  # Replace with the path to your PetImages folder
classes = ['Cat', 'Dog']
train_dir = os.path.join("model_one", 'train')
validation_dir = os.path.join("model_one", 'validation')

# Split ratio
train_ratio = 0.8

# Make new dirs
for cl in classes:
    os.makedirs(os.path.join(train_dir, cl), exist_ok=True)
    os.makedirs(os.path.join(validation_dir, cl), exist_ok=True)

# Move files
for cl in classes:
    # Source dir
    src_dir = os.path.join(base_dir, cl)
    all_files = os.listdir(src_dir)
    
    # Shuffle and split files
    train_files = sample(all_files, int(len(all_files) * train_ratio))
    validation_files = list(set(all_files) - set(train_files))
    
    # Move to train_dir
    for file_name in train_files:
        shutil.move(os.path.join(src_dir, file_name), os.path.join(train_dir, cl))
    
    # Move to validation_dir
    for file_name in validation_files:
        shutil.move(os.path.join(src_dir, file_name), os.path.join(validation_dir, cl))


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)



# Load the base model, pre-trained on ImageNet
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the convolutional base
base_model.trainable = False

# Create a new model on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.3)(x)  # Reduced dropout rate as discussed
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Hyper parameter
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = validation_generator.samples // validation_generator.batch_size
print(f"BatchSize {train_generator.batch_size} and we get epoch {steps_per_epoch}, steps{validation_steps}")

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=100,  # Set a higher epoch and let early stopping manage the stopping
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[early_stopping]
)

# Save the model
model.save('cats_and_dogs_classifier_exp1_auto_stopepoch.h5') #cats_and_dogs_classifier.h5


Found 19966 images belonging to 2 classes.
Found 4993 images belonging to 2 classes.


2024-04-22 13:53:36.883186: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 13:53:36.912502: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fde50ce5380 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-04-22 13:53:36.912531: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


BatchSize 20 and we get epoch 998, steps249
Epoch 1/100
998/998 [==============================] - ETA: 0s - loss: 0.7010 - accuracy: 0.5003

/Users/johnny_hung/Documents/ja_interview_entretien/ja_git/myenv/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:890: UserWarning: Truncated File Read
  warnings.warn(str(msg))


998/998 [==============================] - 768s 769ms/step - loss: 0.7010 - accuracy: 0.5003 - val_loss: 0.6931 - val_accuracy: 0.5008
Epoch 2/100
998/998 [==============================] - 889s 891ms/step - loss: 0.6935 - accuracy: 0.4933 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/100
998/998 [==============================] - 827s 829ms/step - loss: 0.6933 - accuracy: 0.4976 - val_loss: 0.6932 - val_accuracy: 0.4994
Epoch 4/100
998/998 [==============================] - 820s 821ms/step - loss: 0.6933 - accuracy: 0.4990 - val_loss: 0.6931 - val_accuracy: 0.5008
Epoch 5/100
998/998 [==============================] - 828s 830ms/step - loss: 0.6932 - accuracy: 0.4921 - val_loss: 0.6931 - val_accuracy: 0.4998
Epoch 6/100
998/998 [==============================] - 820s 822ms/step - loss: 0.6932 - accuracy: 0.5004 - val_loss: 0.6932 - val_accuracy: 0.5000
